# Value Iteration

## Introduction


The pseudo-code for the algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 1998). 

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The reward $r = r(s, a, s')$ is the expected immediate reward on transition from state $s$ to the next state $s'$ under action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 300px;" align="left" caption="Figure 1"/>

The three problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this environment from the lectures and from your previous lab exercise. The grid squares in the figure are numbered as shown. In all three problems, the following is true: 

**Actions available:** The agent has four possible actions in each grid square. These are _west_, _north_, _south_, and _east_. If the direction of movement is blocked by a wall (for example, if the agent executes action south at grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold, the agent collects the gold. In order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square. 

**Hitting the bomb:** On arrival at a grid square that contains the bomb, the agent activates the bomb. 

** Terminal states:** The game terminates when all gold is collected or when the bomb is activated. 


### Instructions ###
Set parameter $\theta$ to 1 to the power of -10.

Set all initial state values $V(s)$ to zero.

Set $\gamma=1$.

Reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. 

## Deterministic environment

In this part, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment. 

Compute the optimal policy using Value Iteration. 

The array `policy` is an array of strings that specifies an optimal action at each grid location. 

The array `v` is an array of floats that contains the expected return at each grid square (that is, the state value).

In [1]:
import numpy as np

In [2]:
# define a function that used to make step
def make_step(current_state, action):
    # when action == North, get next state
    if action == 'n':
        next_state = current_state + 5
        if next_state >= 25:
            next_state = current_state
    
    # when action == East, get next state
    if action == 'e':
        next_state = current_state + 1
        if next_state % 5 == 0:
            next_state = current_state
    
    # when action == South, get next state and rewards
    if action == 's':
        next_state = current_state - 5
        if next_state < 0:
            next_state = current_state

    # when action == West, get next state and rewards
    if action == 'w':
        next_state = current_state - 1
        if next_state % 5 == 4:
            next_state = current_state
            
    return next_state


# define a function that calculate the max and argmax update value
def update(current_state, probability):
    ret = []
    probability = probability + (1 - probability) / len(actions)
    
    # loop for each action
    for action in actions:
        # main intend action
        a1_next_state = make_step(current_state, action)
        a1 = probability * (rewards[a1_next_state] + gamma * v[a1_next_state])
        
        sub_actions = actions[actions != action]
        for sub_action in sub_actions:
            # the other three random action, and get an expectation(sum)
            a2_next_state = make_step(current_state, sub_action)
            a2 = (1 - probability) / (len(actions)-1) * (rewards[a2_next_state] + gamma * v[a2_next_state])
            a1 += a2
        ret.append(a1)
            
    return np.array(ret).max(), np.array(ret).argmax()


# value iteration algorithm simulation
def value_iteration(probability):
    # value iteration algorithm
    delta = 1
    while delta >= theta:
        delta = 0    
        for state in s:
            current_v = v[state]
            v[state] = update(state, probability)[0]
            delta = np.array([delta, np.abs(current_v-v[state])]).max()

    # get the policy
    policy = []
    for state in s_plus:
        policy.append(actions[update(state, probability)[1]])
    
    return v.reshape(5,5)[::-1], np.array(policy).reshape(5,5)[::-1]


# initial default parameters
theta = 1e-10
gamma = 1
actions = np.array(['n','e','s','w'])
s_plus = np.arange(25)
s = np.delete(s_plus, [18,23])
v = np.random.rand(25)
v[[18,23]] = 0
rewards = np.ones(25) * (-1)
rewards[[18,23]] = [-11, 9]

v, policy = value_iteration(probability = 1)
print(policy)
print(v)

[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]]


## Stochastic environment

In this part, the agent is not always able to execute its actions as intended. With probability 0.8, it moves in the intended direction. With probability 0.2, it moves in a random direction. For example, from grid square 0, if the agent executes action _north_, with probability 0.8, the action will work as intended. But with probability 0.2, the agent's motor control system will move in a random direction (including north). For example, with probability 0.05, it will try to move west (where it will be blocked by the wall and hence remain in grid square 0). Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.



In [3]:
# initial the v 
v = np.random.rand(25)
v[[18,23]] = 0

v, policy = value_iteration(probability = 0.8)
print(policy)
print(v)

[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']]
[[6.04169329 7.28756636 8.61359951 0.         8.69262311]
 [4.86185111 5.99087587 6.37082431 0.         6.46721593]
 [3.67550938 4.69621388 4.99441863 3.2189158  5.10250988]
 [2.48699534 3.40945989 3.66922967 2.64122933 3.78610115]
 [1.35979208 2.19733672 2.42878751 1.57272161 2.55202451]]


## Deterministic environment with two pieces of gold

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this part, the environment is identical to the environment in the first part with the following exception: there is an additional piece of gold on grid square 12. Recall from earlier instructions that the terminal state is reached only when **_any_** gold is collected or when the bomb is activated.

In [4]:
# initial default parameters
s_plus = np.arange(25)
s = np.delete(s_plus, [12,18,23])
v = np.random.rand(25)
v[[12,18,23]] = 0
rewards = np.ones(25) * (-1)
rewards[[12,18,23]] = [9,-11,9]

v, policy = value_iteration(probability = 1)
print(policy)
print(v)

[['e' 'e' 'e' 'n' 'w']
 ['e' 'e' 's' 'n' 'n']
 ['e' 'e' 'n' 'w' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
[[7. 8. 9. 0. 9.]
 [7. 8. 9. 0. 8.]
 [8. 9. 0. 9. 8.]
 [7. 8. 9. 8. 7.]
 [6. 7. 8. 7. 6.]]


## Deterministic environment with two pieces of gold

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this part, the environment is identical to the environment in the first part with the following exception: there is an additional piece of gold on grid square 12. Recall from earlier instructions that the terminal state is reached only when **_all_** gold is collected or when the bomb is activated.

### Wait to be implemented

In this part, unlike the any gold is collected is a terminal state before, it needs to collect all the gold to terminal the game, so need to change the state representation into all situations.
The results is shown in the following.

In [5]:
solution_policy = np.array([['e','e','e','w','w'],['e','e','s','n','n'],['e','e','n','w','w'],
                            ['n','n','n','n','n'],['n','n','n','n','n']])

print(solution_policy)

solution_v = np.array([[14,15,16,7,16],[14,15,16,0,15],[15,16,7,16,15],
                       [14,15,16,15,14],[13,14,15,14,13]]).astype(float)

print(solution_v)

[['e' 'e' 'e' 'w' 'w']
 ['e' 'e' 's' 'n' 'n']
 ['e' 'e' 'n' 'w' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']]
[[14. 15. 16.  7. 16.]
 [14. 15. 16.  0. 15.]
 [15. 16.  7. 16. 15.]
 [14. 15. 16. 15. 14.]
 [13. 14. 15. 14. 13.]]
